# get data

In [1]:
import os
import random
import librosa
import numpy as np
import pandas as pd
filenames = []
dir = "data/mini_speech_commands"
labels = os.listdir(dir)
labels.remove('README.md')
labels.remove('.DS_Store')

# for root,dirs,files in os.walk(dir):
#     if root != dir:
#         for file in files:
#             label = root.split("/")[-1]
#             file_path = root + "/" + file
#             wave, sr = librosa.load(file_path, mono=True)
#             mfcc = librosa.feature.mfcc(wave, sr)
#             mfcc_pd = pd.DataFrame(mfcc)
#             mfcc_pd = mfcc_pd.apply(np.average,axis=1)

for root,dirs,files in os.walk(dir):
    if root != dir:
        for file in files:
            label = root.split("/")[-1]
            filename = root + "/" + file
            filenames.append(filename)
random.shuffle(filenames)
labels

['right', 'go', 'no', 'left', 'stop', 'up', 'down', 'yes']

# split datasets

In [2]:
train_files = filenames[0:int(len(filenames)*0.8)]
test_files = filenames[int(len(filenames)*0.8):]

In [3]:
def getLabelID(file,labels):
    label = file.split("/")[-2]
    labelID = labels.index(label)
    return labelID
    

In [4]:
def getMFCC(file):
    wave, sr = librosa.load(file, mono=True)
    mfcc = librosa.feature.mfcc(wave, sr)
    mfcc_pd = pd.DataFrame(mfcc)
    mfcc_pd = mfcc_pd.apply(np.average,axis=1)
    mfcc = mfcc_pd.to_list()
    return mfcc

In [5]:
def getX_Y(filenames):
    Xs = []
    Ys = []
    for filename in filenames:
        labelID = getLabelID(filename,labels)
        mfcc = getMFCC(filename)
        Xs.append(mfcc)
        Ys.append(labelID)
    return Xs,Ys   

In [6]:
train_Xs,train_Ys = getX_Y(train_files)

In [7]:
test_Xs,test_Ys =getX_Y(test_files)

In [8]:
A = np.array(train_Xs)
B = np.reshape(np.array(train_Ys),(len(train_Ys),1))


In [9]:
train_dataset = pd.DataFrame(np.hstack((A,B)),columns=[i for i in range(20)]+['label'])
train_dataset.to_csv('sd.csv',index=False)

In [13]:
group = train_dataset.groupby('label')
train_dataset_mean = group.mean()
train_dataset_sd = group.std()
train_dataset_count = group.count()
train_dataset_count = train_dataset_count.apply(np.average,axis=1)
train_dataset_mean

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
label,,,,,,,,,,,,,,,,,,,,
0.0,-438.446033,109.512868,-21.245432,28.438246,-9.747272,9.172969,-11.052589,-0.272818,-3.415165,-2.910535,-0.047181,-6.567383,2.496684,-5.306148,2.173034,-4.517760,-0.889630,-2.255635,-3.041102,-0.891726
1.0,-441.230695,111.239424,-10.060371,22.226564,-13.175000,5.811673,-5.406767,0.422098,-4.749581,-3.241482,-0.352431,-4.350418,1.597116,-5.656364,2.540401,-3.809689,-1.492384,-2.693873,-3.108798,-1.057646
2.0,-442.575832,118.728026,-10.312961,26.049813,-11.115763,5.162228,-5.360380,1.651773,-5.254059,-3.070262,0.639147,-6.778133,0.506989,-6.348589,0.269694,-5.264111,-2.705411,-3.564850,-3.990552,-1.842095
3.0,-422.022396,98.733463,-21.806685,31.124582,-17.813870,11.988598,-10.130333,-0.243955,-2.986017,0.247615,3.008772,-7.421883,2.362361,-5.541517,3.360155,-5.008653,-1.234927,-2.132492,-2.707942,-0.955296
4.0,-431.963330,94.874473,-23.046580,25.641025,-26.079338,15.683786,-8.370328,4.839053,-3.733247,-1.622373,3.976240,-5.606314,3.489083,-7.003874,3.064549,-5.485844,-0.449693,-2.614371,-2.050882,-1.232105
5.0,-461.322310,102.333495,-17.360410,18.075264,-16.699497,8.811318,-2.188288,2.605477,-2.937901,-0.776649,4.375281,-3.493115,1.824004,-5.605647,0.866128,-4.313636,-1.685909,-1.678018,-2.511541,-1.024066
6.0,-424.614968,118.729678,-25.704126,28.019585,-11.886352,9.722861,-6.966360,3.040351,-3.053751,-1.420949,1.387414,-7.872721,1.326215,-6.929239,0.122330,-6.239762,-2.382303,-3.845719,-3.521140,-1.421213
7.0,-426.892640,79.608770,-20.199048,45.142039,-20.793992,18.184928,-17.356928,4.860562,-2.899019,-1.326800,2.177306,-8.727152,4.624003,-7.391289,4.242099,-6.806578,0.085493,-3.054420,-2.843949,-1.393050


In [14]:
import math

In [31]:
def predict(mean_dataset,sd_dataset,count_dataset,X):
    probs = []
    mean_dataset = np.array(mean_dataset)
    sd_dataset = np.array(sd_dataset)
    count_dataset = np.array(count_dataset)
    for i in range(mean_dataset.shape[0]):
        cumu_prob = 1
        for j in range(mean_dataset.shape[1]):
            x = (X[j] - mean_dataset[i][j])/sd_dataset[i][j]
            cumu_prob *= (1/math.sqrt(2*math.pi))*np.exp(-x**2/(2))
            # cumu_prob *= (1/(math.sqrt(2*math.pi)*sd_dataset[i][j]))*np.exp(-(X[j]-mean_dataset[i][j])**2/(2*sd_dataset[i][j]**2))
        cumu_prob *= count_dataset[i]/sum(count_dataset)
        probs.append(cumu_prob)

    label = probs.index(max(probs))
    return label

a = predict(train_dataset_mean,train_dataset_sd,train_dataset_count,train_Xs[1])

In [32]:
predict_Ys = []
for X in test_Xs:
    predict_Ys.append(predict(train_dataset_mean,train_dataset_sd,train_dataset_count,X))
sum(np.array(test_Ys)==np.array(predict_Ys))/len(test_Ys)

0.35875

In [29]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(np.array(train_Xs), np.array(train_Ys)).predict(np.array(test_Xs))
sum(y_pred == np.array(test_Ys))/len(test_Ys)

0.346875